In [4]:
# ! pip install --upgrade tensorflow-hub

In [1]:
import cv2
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt 
import tensorflow_hub as hub
import imutils

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [2]:
# Loading model 

model = hub.load("https://tfhub.dev/google/movenet/singlepose/lightning/4")

movenet = model.signatures['serving_default']

In [3]:
# Drawing circles on predictions 

def draw_circels(frame, outputs):
    
    keypoints = np.squeeze(outputs['output_0'].numpy())*[frame.shape[0],frame.shape[1],1]

    for kp in keypoints:
        
        ky,kx,kp_conf = kp

        if kp_conf > 0.6:

            cv2.circle(frame,(int(kx),int(ky)),4,(0,255,0),-1)

In [4]:
EDGE_LIST = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}



def draw_edges(frame, outputs, edges):

    keypoints = np.squeeze(outputs['output_0'].numpy())*[frame.shape[0],frame.shape[1],1]

    for edge,color in edges.items():
        p1,p2 = edge
        y1,x1,c1 = keypoints[p1]
        y2,x2,c2 = keypoints[p2]

        if (c1>0.6) & (c2>0.6):
            cv2.line(frame,(int(x1),int(y1)),(int(x2),int(y2)), (255,0,0), 2)


In [6]:
# Making predictions 

cap = cv2.VideoCapture(0)

while cap.isOpened():

    ret,frame = cap.read()

    img = frame.copy()

    # resizing frame
    input_image = tf.image.resize_with_pad(np.expand_dims(img, axis=0),192,192)
    input_image = tf.cast(input_image,dtype=tf.int32)

    # run model

    outputs = movenet(input_image)

    
    # plotting ouputs 

    draw_circels(frame, outputs)
    draw_edges(frame, outputs, EDGE_LIST)
    

    cv2.imshow("Movenet Detection",frame)

    if cv2.waitKey(10) & 0xFF==ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
